### 7. 서울시 특정 행정동 치킨외식업체 리뷰 데이터수집

In [12]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# 403 forbidden (접근거부 에러) 해결 옵션
options = webdriver.ChromeOptions()
options.add_argument("no-sandbox")
options.add_argument("disable-gpu")
options.add_argument("--lang=ko_KR")
options.add_argument('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.95 Safari/537.36')

keyword = input("검색 동 : ")

link = pd.read_csv('./data/'+keyword+" 치킨집 카카오맵 데이터 크롤링.csv")
page_urls = []
page_urls = link['링크주소']
columns = ['score', 'review']

# Chrome 웹드라이버 설치 경로
excutable_path = './data/chromedriver.exe'
driver = webdriver.Chrome(executable_path=excutable_path, chrome_options=options)
source_url = "https://map.kakao.com/"
# 데이터프레임 생성
df = pd.DataFrame(columns=columns)

print(keyword, '치킨집(카카오맵) 리뷰 데이터 수집 시작')
print()

for page_url in page_urls: # url 갯수 만큼(103개) 반복

    # 각 업체 url 접속
    driver.get(page_url)
    time.sleep(2)

    # 첫 페이지 추출
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents = soup.find(name="div", attrs={"class":"evaluation_review"})

    # 별점 추출
    rates = contents.find_all(name="em", attrs={"class":"num_rate"})

    # 리뷰 추출
    reviews = contents.find_all(name="p", attrs={"class":"txt_comment"})

    # 별점, 리뷰 리스트 값 한개씩 붙혀서 row(series)로 구성 후 
    # 데이터 프레임에 한줄씩 붙힘
    for rate, review in zip(rates, reviews):
        row = [rate.text[0], review.find(name="span").text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)

    # 2~5 페이지 까지 진행
    for button_num in range(2, 6):
        # 다음 페이지 버튼이 없는경우 try에서 빠져 나가도록
        try:
            more_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            more_reviews.click()
            time.sleep(2)

            # 각 페이지 추출
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents = soup.find(name="div", attrs={"class":"evaluation_review"})

            # 별점 추출
            rates = contents.find_all(name="em", attrs={"class":"num_rate"})

            # 리뷰 추출
            reviews = contents.find_all(name="p", attrs={"class":"txt_comment"})

            # 별점, 리뷰 리스트 값 한개씩 붙혀서 row(series)로 구성 후 
            # 데이터 프레임에 한줄씩 붙힘
            for rate, review in zip(rates, reviews):
                row = [rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=df.columns)
                df = df.append(series, ignore_index=True)
        except:
            break    

savepath = "./data/"
df.to_csv(savepath+keyword+" 치킨집 카카오맵 리뷰 데이터 크롤링.csv", index = False, mode='w', encoding = 'utf-8-sig')
print(keyword+' 완료'+'================')
print()
driver.close()

검색 동 : 묵1동
묵1동 치킨집(카카오맵) 리뷰 데이터 수집 시작

묵1동 완료================

